In [1]:
import numpy as np
import cv2

In [2]:
dragon = cv2.imread('./test-images/wenwen.jpg')
dragon.shape

(166, 163, 3)

In [3]:
def Expand(channel,block_size):
    Height = channel.shape[0]
    Width = channel.shape[1]
    Max = Height if Height > Width else Width
    if(Height % block_size == 0 and Width % block_size == 0):
        return channel
    else:
        # 计算扩充块的大小
        if(Max % block_size == 0): N = Max
        else: N = block_size * ((int)(Max / block_size) + 1)
        expand_channel = np.zeros((N,N))
        print(expand_channel.shape)
        for i in range(Height):
            for j in range(Width):
                expand_channel[i][j] = channel[i][j]
        return expand_channel

In [4]:
channel = dragon[:,:,0]
channel.shape
channel

array([[204, 203, 200, ..., 185, 185, 185],
       [202, 203, 201, ..., 185, 185, 185],
       [199, 200, 201, ..., 186, 186, 186],
       ...,
       [ 70,  68,  69, ...,  77,  77,  77],
       [ 70,  68,  68, ...,  71,  70,  69],
       [ 70,  69,  67, ...,  77,  77,  76]], dtype=uint8)

In [5]:
expand = Expand(channel,16)
expand.shape

(176, 176)


(176, 176)

In [6]:
cv2.imwrite('./test-images/expand-wenwen.jpg',expand)

True

In [7]:
def Divide(channel,block_size):
    Height = channel.shape[0]
    Width = channel.shape[1]
    if Height != Width: return 
    else: 
        num = (int) (Height / block_size)
        train_data = np.zeros((num*num,block_size*block_size))
        # 按块处理
        for i in range(num):
            for j in range(num):
                    block = channel[j * block_size : (j + 1) * block_size, 
                                    i * block_size : (i + 1) * block_size].flatten()
                    col = (int)(i * num + j)
                    train_data[col,:] = block
        return train_data / 255.0

In [8]:
def Reback(train_data,block_size,num,Height,Width):
    width = height = block_size * num # 扩充后的宽高
    # 还原
    expand_channel = np.zeros((width,height))
    for i in range(num):
        for j in range(num):
            block = train_data[i * num + j, :].reshape(block_size,block_size)
            expand_channel[j * block_size : (j + 1) * block_size, 
                    i * block_size : (i + 1) * block_size] = block
    # 去除扩充后的数据
    channel = np.zeros((Height,Width))
    for i in range(Height):
        for j in range(Width):
           channel[i][j] = expand_channel[i][j]
    return channel * 255.0

In [9]:
train_data = Divide(expand,16)
train_data.shape

(121, 256)

In [10]:
r = Reback(train_data,16,11,166,163)

In [11]:
train_data[0,16:32]*255,r[1,0:16]

(array([202., 203., 201., 200., 198., 197., 197., 195., 194., 193., 179.,
        157., 139., 136., 157., 180.]),
 array([202., 203., 201., 200., 198., 197., 197., 195., 194., 193., 179.,
        157., 139., 136., 157., 180.]))

In [12]:
sum(((np.array([[[1,2],[2,2]],
          [[1,1],[1,1]]]) - np.array([[[1,1],[1,1]],
                                      [[2,2],[2,2]]])).flatten()) ** 2)

7

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import cv2

In [14]:
img = cv2.imread('./test-images/wenwen.jpg')

In [15]:
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data  = scaler.fit(train_data)
mlp = MLPRegressor(hidden_layer_sizes=(100,50,20),max_iter=500)
mlp.fit(train_data,train_data)
predict = mlp.predict(train_data)

C:\Users\hp\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


In [16]:
train_data

array([[ 1.1431777 ,  1.14512206,  1.12523821, ...,  1.23944843,
         1.32543837,  1.26636011],
       [ 1.08254895,  1.09998369,  1.15570109, ...,  1.3680153 ,
         1.32543837,  1.33114515],
       [ 1.23412084,  1.20530656,  1.06431246, ...,  1.1623083 ,
         1.32543837,  1.48662924],
       ...,
       [-0.06939738, -0.20902923, -0.96146878, ..., -1.31903239,
        -1.32458784, -1.33799844],
       [-0.84241399, -0.87105875, -0.93100591, ..., -1.31903239,
        -1.32458784, -1.33799844],
       [-0.59989897, -0.67545912, -0.8091544 , ..., -1.31903239,
        -1.32458784, -1.33799844]])

In [17]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(predict,train_data)
mse

0.200015825399466

In [18]:
predict,train_data

(array([[ 1.11444005,  0.73772489,  1.02281497, ...,  0.73805523,
          1.23729837,  0.83152273],
        [ 0.99638383,  1.12673055,  0.94107374, ...,  1.2581879 ,
          1.13360079,  1.03559982],
        [ 0.33338885,  0.19472625,  0.21973767, ...,  0.98189032,
          0.45322551,  1.0348922 ],
        ...,
        [-0.78110518, -0.98231086, -0.65485159, ..., -1.44505588,
         -1.42236056, -1.25851745],
        [-0.71097569, -1.03795481, -0.61293092, ..., -1.4350371 ,
         -1.32380592, -1.1927388 ],
        [-0.96731143, -0.91847836, -0.92474328, ..., -1.6656094 ,
         -1.57032803, -1.29900505]]),
 array([[ 1.1431777 ,  1.14512206,  1.12523821, ...,  1.23944843,
          1.32543837,  1.26636011],
        [ 1.08254895,  1.09998369,  1.15570109, ...,  1.3680153 ,
          1.32543837,  1.33114515],
        [ 1.23412084,  1.20530656,  1.06431246, ...,  1.1623083 ,
          1.32543837,  1.48662924],
        ...,
        [-0.06939738, -0.20902923, -0.96146878, ..., -

In [19]:
r = Reback(train_data,16,11,166,163)

In [20]:
r

array([[ 291.51031459,  292.00612585,  286.93574385, ...,  218.07373632,
         222.94440991,  228.67549381],
       [ 290.14820967,  301.28781071,  297.39765431, ...,  223.19343434,
         231.66395376,  236.04195751],
       [ 290.26341874,  305.97813707,  309.45718695, ...,  237.81862691,
         249.29321098,  250.44018639],
       ...,
       [-231.03412387, -232.0334432 , -209.45086707, ..., -202.24831879,
        -195.35936197, -177.87011329],
       [-223.25242866, -230.7129077 , -239.56474039, ..., -219.16442263,
        -222.5645534 , -235.42310107],
       [-204.43833374, -230.77378995, -243.19270344, ..., -176.8124134 ,
        -197.88860981, -205.90224244]])

In [21]:
cv2.imwrite('./test-images/test-wenwen.jpg',r)

True